In [13]:
# Ensure Python's built-in int() and float() override Sage's types
from builtins import int, float
import requests
import pandas as pd
from datetime import datetime
import os

# Ensure data folder exists
os.makedirs('../data', exist_ok=True)

### A. Fetch Historical Data (last 365 days)
hist_url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
hist_params = {'vs_currency': 'usd', 'days': '60'}

hist_response = requests.get(hist_url, params=hist_params)
if hist_response.status_code == 200:
    data = hist_response.json()
    prices_raw = data['prices']

    # Explicitly cast values to Python float and int to avoid Sage Rational error
    timestamps = [datetime.fromtimestamp(float(p[0]) / 1000) for p in prices_raw]
    prices = [float(p[1]) for p in prices_raw]

    df_hist = pd.DataFrame({'timestamp': timestamps, 'price_usd': prices})
    df_hist.to_csv('../data/bitcoin_historical.csv', index=False)
    print(f"✅ Historical data saved: {len(df_hist)} rows")
else:
    print(f"❌ Failed to fetch historical data: {hist_response.status_code}")

### B. Fetch Live Price
live_url = 'https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd'
live_response = requests.get(live_url)

if live_response.status_code == 200:
    price = float(live_response.json()['bitcoin']['usd'])
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df_live = pd.DataFrame([{'timestamp': timestamp, 'price_usd': price}])

    live_path = '../data/bitcoin_live.csv'
    if os.path.exists(live_path):
        df_live.to_csv(live_path, mode='a', header=False, index=False)
    else:
        df_live.to_csv(live_path, index=False)

    print(f"✅ Live price saved: {price} USD")
else:
    print(f"❌ Failed to fetch live price: {live_response.status_code}")


❌ Failed to fetch historical data: 429
❌ Failed to fetch live price: 429
